# Neighbourhood Segmentation and Clustering Project : Toronto

In the following notebook we will be segmenting and clustering neighbourhoods across Toronto, Canada. The data to be used for this will be obtained from a Wikipedia table through scraping using the Beautiful Soup library.

## 1. Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [62]:
## import the library we use to open URLs
import urllib.request

In [63]:
# import the BeautifulSoup library to parse HTML and XML documents
from bs4 import BeautifulSoup

## 2. Webscraping and the creation of the initial dataframe

In [64]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [65]:
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

In [66]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

After this I used Beautiful Soup’s prettify function to view the HTML code directly on the notebook

In [67]:
# Use Beautiful Soup’s prettify function to view the HTML code directly on the notebook
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"YByV1ApAAEEAAFgXy2gAAAAL","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1004901187,"wgRevisionId":1004901187,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communications in Ont

By looking at the Wikipedia page for the Canadian postal codes,there is a quite a bit of information to churn through. 

What we are looking for though is the table which shows the postal codes for each borough within Toronto. So we search through for it in the HTML code. We find it starting with an HTML table tag and a class identifier of ”wikitable sortable".

Scrolling down we see how the table is made up and it's evident the rows start and end with 'tr' and '/tr' tags.
The top row of headers has 'th' tags while the data rows beneath for each borough has 'td' tags. It’s in these 'td' tags that we will tell Python to extract our data from.

We know the data resides within an HTML table so we send Beautiful Soup off to retrieve all instances of the table tag within the page and add them to an array called all_tables.

In [68]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")
all_tables

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighbourhood
 </th></tr>
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>
 <tr>
 <td>M2B

Looking through the output of ”all_tables” we can see that the class id of our chosen table is ”wikitable sortable”. We can use this to get BS to only bring back the table data for this particular table and keep that in a variable called ”right_table“:

In [69]:
right_table=soup.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

We know that the table is set up in rows (starting with 'tr' tags) with the data sitting within 'td' tags in each row. We aren’t interested in the header row with the 'th' elements as we know what each of the columns represent by looking at the table.

We loop through the rows to get the data for every postal code in the table. The table is well structured with each code having it’s own defined row.There are three columns in the table that we want to scrape the data from so we will set up three empty lists (A, B, C) to store our data in.

In [70]:
A=[]
B=[]
C=[]

We use the Beautiful Soup ‘find_all’ function again and set it to look for the string ‘tr’. We will then set up a FOR loop for each row within that array and set Python to loop through the rows, one by one. 

In [71]:
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

We import pandas and create a dataframe assigning each of the lists A-C into a column with the name of our source table columns i.e. Postal Code, Borough and Neighbourhood.

In [72]:
import pandas as pd
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

df

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
...,...,...,...
175,M5Z\n,Not assigned\n,Not assigned\n
176,M6Z\n,Not assigned\n,Not assigned\n
177,M7Z\n,Not assigned\n,Not assigned\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."


## 3. Modifying the dataframe

### 3.1 Remove cells that have no borough assigned

In [73]:
df.drop(df.loc[df['Borough']=="Not assigned\n"].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [74]:
df

,Postal Code,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods\n
1,M4A\n,North York\n,Victoria Village\n
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"
...,...,...,...
98,M8X\n,Etobicoke\n,"The Kingsway, Montgomery Road, Old Mill North\n"
99,M4Y\n,Downtown Toronto\n,Church and Wellesley\n
100,M7Y\n,East Toronto\n,"Business reply mail Processing Centre, South C..."
101,M8Y\n,Etobicoke\n,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


We need to edit the information within all the rows that have'\n' after the postal code, borough and neighbourhood names. For this we use the string replace function.

### 3.2. Check if there are any neighborhoods that still aren't assigned 

In [75]:
df[df['Neighborhood'] == 'Not assigned']

,Postal Code,Borough,Neighborhood


### Removing the unwated characters within the data strings 

In [80]:
df['Borough'] = df['Borough'].str.replace('(\n)', '')
df['Postal Code'] = df['Postal Code'].str.replace('(\n)', '')
df['Neighborhood'] = df['Neighborhood'].str.replace('(\n)', '')


In [81]:
df

,Postal Code,Borough,Neighborhood,Neighbourhood
0,M3A,North York,Parkwoods,Parkwoods
1,M4A,North York,Victoria Village,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights","Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North","The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...","Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...","Old Mill South, King's Mill Park, Sunnylea, Hu..."


### 3.3 Check the shape of the dataset 

In [82]:
df.shape

(103, 4)

# 4. Adding in the geolocation data

In [83]:
coordsdf = pd.read_csv("Geospatial_Coordinates.csv")

In [84]:
coordsdf

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [85]:
#Filtering out the locations by postal code
dfcodes = df[df['Postal Code'].isin(coordsdf['Postal Code'].tolist())]
#Then merging it to create a new table with the latitudes and longitudes of each borough and neighbourhood
newcoordsdf = pd.merge(dfcodes, coordsdf)


In [128]:
newcoordsdf

,Postal Code,Borough,Neighborhood,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale","The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West","India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park","North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East","Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...","Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [134]:
#extracting only the rows that are within the Toronto region 
newcoordsdf = newcoordsdf[newcoordsdf['Borough'].str.contains('Toronto')].reset_index(drop=True)
newcoordsdf.sort_values(by=['Postal Code'], inplace=True)
newcoordsdf.reset_index(drop=True, inplace=True)

print(f'Shape of final dataframe: {newcoordsdf.shape}')

Shape of final dataframe: (39, 6)


# 5. Clustering & Segmenting the Neighborhoods

In [135]:
#declaring a new variable to work with for clustering
torontodata = newcoordsdf.drop(columns={'Postal Code'}, axis=1)
torontodata.head()

,Borough,Neighborhood,Neighbourhood,Latitude,Longitude
0,East Toronto,The Beaches,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale","The Danforth West, Riverdale",43.679557,-79.352188
2,East Toronto,"India Bazaar, The Beaches West","India Bazaar, The Beaches West",43.668999,-79.315572
3,East Toronto,Studio District,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,Lawrence Park,43.728020,-79.388790


## 5.1 Using the Four Square API to obtain venue data for feature engineering

In [136]:
CLIENT_ID = 'W3H4FHMKKTKSNPZNLVFXKE2HQORWZ1NOH0ASFNPPSCH353BA' # Foursquare ID
CLIENT_SECRET = 'IBB4EYDQXH1PZSCFNU5KQYDYO3QSXGX0JOGEYVSFWRZO1DVW' # Foursquare Secret
ACCESS_TOKEN = 'RURAHMIHUCDG21JRXN5ZUP2D2EP4MOQUPDBR3SHS3PGLR0W1' # FourSquare Access Token
VERSION = '20200501'
LIMIT = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W3H4FHMKKTKSNPZNLVFXKE2HQORWZ1NOH0ASFNPPSCH353BA
CLIENT_SECRET:IBB4EYDQXH1PZSCFNU5KQYDYO3QSXGX0JOGEYVSFWRZO1DVW


In [137]:
#getting a list of all the venues thar are in neighbourhoods around Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [139]:
# Get nearby venues of all neighborhoods 
toronto_venues = getNearbyVenues(names=torontodata['Neighborhood'], 
                                 latitudes=torontodata['Latitude'], 
                                 longitudes=torontodata['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [140]:
print('A list of all nearby venues in Toronto with the neighborhood that they are in:')
toronto_venues

A list of all nearby venues in Toronto with the neighborhood that they are in:


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
...,...,...,...,...,...,...,...
1589,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
1590,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
1591,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
1592,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,ONE Academy,43.662253,-79.326911,Gym / Fitness Center


In [141]:
#determining the total number of unique venue categories 
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


## 5.2 Creating a one hot vector for the mean frequency of each category in each neighborhood 

In [142]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let us look at the new shape of the data frame 

In [143]:
toronto_onehot.shape

(1594, 233)

In [144]:
# Group and average the frequency of unique category in each neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
4,Central Bay Street,0.016393,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.016393,0.0,0.0,0.016393


In [98]:
toronto_grouped.shape

(39, 233)

## 5.3 Applying K Means to the engineered feature set

We will now cluster the neighbourhoods. We will do this by running 'k means'to cluster the neighbourhoods into 5 clusters 

In [201]:
# Set up k and run KMean
k = 4
km = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped.drop(columns={'Neighborhood'}, axis=1))

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 2, 0, 0, 0, 3, 0], dtype=int32)

In [202]:
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
4,Central Bay Street,0.016393,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.016393,0.0,0.0,0.016393


# 6. Characterising the Clusters

## 6.1 Obtaining a list of the most common venues

In [147]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [148]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Farmers Market,Cheese Shop,Restaurant,Beer Bar,Café,Concert Hall
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Climbing Gym,Burrito Place,Restaurant,Italian Restaurant,Intersection,Stadium,Bar
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Pizza Place,Restaurant,Burrito Place,Fast Food Restaurant,Auto Workshop,Farmers Market,Spa,Comic Shop,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Plane,Rental Car Location,Harbor / Marina,Bar,Coffee Shop,Airport Gate
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Bubble Tea Shop,Burger Joint,Salad Place,Poke Place,Portuguese Restaurant,Ramen Restaurant


In [149]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with torontodata to add latitude/longitude for each neighborhood
toronto_merged = torontodata

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,The Beaches,43.676357,-79.293031,0,Park,Trail,Pub,Health Food Store,Wine Bar,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store
1,East Toronto,"The Danforth West, Riverdale","The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Bubble Tea Shop,Bakery,Pub,Pizza Place
2,East Toronto,"India Bazaar, The Beaches West","India Bazaar, The Beaches West",43.668999,-79.315572,0,Fast Food Restaurant,Sushi Restaurant,Movie Theater,Sandwich Place,Brewery,Restaurant,Italian Restaurant,Ice Cream Shop,Steakhouse,Fish & Chips Shop
3,East Toronto,Studio District,Studio District,43.659526,-79.340923,0,Coffee Shop,Gastropub,Bakery,Brewery,Café,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Cheese Shop
4,Central Toronto,Lawrence Park,Lawrence Park,43.728020,-79.388790,0,Bus Line,Park,Swim School,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


### Cluster One Analysis 

In [162]:
One = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [183]:
One[['1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue']].mode()



,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Coffee Shop,Sushi Restaurant,Café


### Cluster Two Analysis

In [184]:
Two = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [185]:
Two

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,"Parkdale, Roncesvalles",1,Gift Shop,Breakfast Spot,Cuban Restaurant,Bookstore,Dog Run,Bar,Italian Restaurant,Dessert Shop,Restaurant,Movie Theater


Only consists of one neighborhood so there is not enough data to infer 

### Cluster Three Analysis 

In [189]:
Three = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [190]:
Three

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Bay Street,2,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Bubble Tea Shop,Burger Joint,Salad Place,Poke Place,Portuguese Restaurant,Ramen Restaurant
31,"Dufferin, Dovercourt Village",2,Pharmacy,Bakery,Pet Store,Supermarket,Park,Music Venue,Pool,Middle Eastern Restaurant,Café,Brewery


Majority of the venues in this cluster are eateries and restaurants so cluster three can fall primarily in this category.

### Cluster Four Analysis

In [192]:
Four = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [193]:
Four

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Davisville,3,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Café,Italian Restaurant,Sushi Restaurant,Gym,Greek Restaurant,Restaurant


Majority of the venues in this cluster are eateries and restaurants so cluster three can fall primarily in this category.

### Final Comments 

Based on the anaysis of all the common venues within each cluster we can conclude that: 
- Cluster 1 : This is primarily an eatery based cluster with numerous restaurants and cafe's
- Cluster 2 : Consists of a single neighbourhood so venues are varied
- Cluster 3 : This is also an eatery cluster with numerous cafes and light meal joints 
- Cluster 4 : Consists of a single neighbourhood albeit with numerous eateries especially restaurants

## 6.2 Viewing the clusters on a map

In [195]:
#creating a new variable to hold the map data
df2 = newcoordsdf

In [196]:
df2

,Postal Code,Borough,Neighborhood,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale","The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West","India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park","North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East","Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...","Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [197]:
# Add cluster labels to the new df
df2.insert(1, 'Cluster', kmeans.labels_)
df2.head()

,Postal Code,Cluster,Borough,Neighborhood,Neighbourhood,Latitude,Longitude
0,M4E,0,East Toronto,The Beaches,The Beaches,43.676357,-79.293031
1,M4K,0,East Toronto,"The Danforth West, Riverdale","The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,0,East Toronto,"India Bazaar, The Beaches West","India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,0,East Toronto,Studio District,Studio District,43.659526,-79.340923
4,M4N,2,Central Toronto,Lawrence Park,Lawrence Park,43.728020,-79.388790


In [198]:
# Generate the latitude and longitude of Toronto
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [199]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df2['Latitude'], df2['Longitude'], df2['Neighbourhood'], df2['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters